In [1]:
import os
import datetime
import pandas as pd
from client import client
    

In [2]:
def try_create_directory(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory failed")
    else:
        print ("Directory created successfully")

try_create_directory(f'data')
try_create_directory(f'data/by-date')
try_create_directory(f'data/by-date-range')


Creation of the directory failed
Creation of the directory failed
Creation of the directory failed


-------------------------------------------------------
Countries

In [3]:
countries = client.get_countries()
data = {'Name': [country['name'] for country in countries]}
df = pd.DataFrame(data, columns=['Name'])
df.to_csv ('data/countries.csv', index = False, header=True)

---------------------------------
Country's today statisctics

In [4]:
def generate_and_save_country_last_stats(country, save=True):
    country_data = client.get_country_detail(country)
    lastUpdate = country_data[0]['lastUpdate']
    date = datetime.datetime.fromtimestamp(lastUpdate/1000.0)

    data = {
        'provinceState': [country_data[i]['provinceState'] for i in range(len(country_data))],
        'Confirmed': [country_data[i]['confirmed'] for i in range(len(country_data))],
        'Recovered': [country_data[i]['recovered'] for i in range(len(country_data))],
        'Deaths': [country_data[i]['deaths'] for i in range(len(country_data))],
        'Active': [country_data[i]['active'] for i in range(len(country_data))],
        'date': [date]
    }
    if save:
        df = pd.DataFrame(data, columns=['provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date/{date}')
        df.to_csv (f'data/by-date/{date}/{country}.csv', index = False, header=True)
        
    return data

In [82]:
country = "Kazakhstan"
_ = generate_and_save_country_last_stats(country)

Creation of the directory failed


_____
Statistics of specific country for given date

In [6]:
def generate_and_save_country_stats_by_date(country, day, month, year, save=True):
    country_data = client.get_country_stats_by_date(country, day, month, year)
    lastUpdate = country_data[0]['lastUpdate']
    date = datetime.datetime.strptime(lastUpdate, '%Y-%m-%d %H:%M:%S')

    data = {
        'provinceState': [country_data[i]['provinceState'] for i in range(len(country_data))],
        'Confirmed': [country_data[i]['confirmed'] for i in range(len(country_data))],
        'Recovered': [country_data[i]['recovered'] for i in range(len(country_data))],
        'Deaths': [country_data[i]['deaths'] for i in range(len(country_data))],
        'Active': [country_data[i]['active'] for i in range(len(country_data))],
        'date': [date] * len(country_data)
    }

    if save:
        df = pd.DataFrame(data, columns=['provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date/{date}')
        df.to_csv (f'data/by-date/{date}/{country}.csv', index = False, header=True)
    return data

In [7]:
country = "China"
year = 2021
month = 1
day = 25
_ = generate_and_save_country_stats_by_date(country, day, month, year)

Creation of the directory failed


___________
Statisctics of world situation for given date

In [8]:

def generate_and_save_world_stats_by_date(day, month, year, save=True):
    world_data = client.get_world_stats_by_date(day, month, year)
    lastUpdate = world_data[0]['lastUpdate']
    date = datetime.datetime.strptime(lastUpdate, '%Y-%m-%d %H:%M:%S')

    data = {
        'country': [world_data[i]['countryRegion'] for i in range(len(world_data))],
        'provinceState': [world_data[i]['provinceState'] for i in range(len(world_data))],
        'Confirmed': [world_data[i]['confirmed'] for i in range(len(world_data))],
        'Recovered': [world_data[i]['recovered'] for i in range(len(world_data))],
        'Deaths': [world_data[i]['deaths'] for i in range(len(world_data))],
        'Active': [world_data[i]['active'] for i in range(len(world_data))],
        'date': [date] * len(world_data)
    }
    
    if save:
        df = pd.DataFrame(data, columns=['country', 'provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date/{date}')
        try_create_directory(f'data/by-date/{date}/world')
        df.to_csv (f'data/by-date/{date}/world/World.csv', index = False, header=True)

    return data
        

In [9]:
year = 2021
month = 1
day = 25
_ = generate_and_save_world_stats_by_date(day, month, year)

Creation of the directory failed
Creation of the directory failed


_____
Statistics of country for specific period of time

In [23]:
def generate_and_save_contry_stats_in_date_range(country, date1, date2, save=True):
    if len(date1) != 3 or len(date2) != 3:
        raise Exception("Incorrect format of date.")

    start_date = datetime.date(*reversed(date1))
    end_date = datetime.date(*reversed(date2))
    day_count = (end_date - start_date).days + 1

    if day_count <= 0:
        raise Exception("Start date must be before end date.")
    
    data = {
        'provinceState': [],
        'Confirmed': [],
        'Recovered': [],
        'Deaths': [],
        'Active': [],
        'date': []
    }

    for date in (start_date + datetime.timedelta(n) for n in range(day_count)):
        try:
            date_data = generate_and_save_country_stats_by_date(country, date.day, date.month, date.year, save=False)
            for key in data.keys():
                data[key] += date_data[key]
            

        except Exception as e:
            print(e)

    if save:
        df = pd.DataFrame(data, columns=['provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date-range/{start_date} --> {end_date}')
        df.to_csv (f'data/by-date-range/{start_date} --> {end_date}/{country}.csv', index = False, header=True)
    return data


In [25]:
country = "China"
date1 = (1,6,2020)
date2 = (4,6,2020)

_ = generate_and_save_contry_stats_in_date_range(country, date1, date2)

Directory created successfully


{'provinceState': ['Anhui',
  'Beijing',
  'Chongqing',
  'Fujian',
  'Gansu',
  'Guangdong',
  'Guangxi',
  'Guizhou',
  'Hainan',
  'Hebei',
  'Heilongjiang',
  'Henan',
  'Hong Kong',
  'Hubei',
  'Hunan',
  'Inner Mongolia',
  'Jiangsu',
  'Jiangxi',
  'Jilin',
  'Liaoning',
  'Macau',
  'Ningxia',
  'Qinghai',
  'Shaanxi',
  'Shandong',
  'Shanghai',
  'Shanxi',
  'Sichuan',
  'Tianjin',
  'Tibet',
  'Xinjiang',
  'Yunnan',
  'Zhejiang',
  'Anhui',
  'Beijing',
  'Chongqing',
  'Fujian',
  'Gansu',
  'Guangdong',
  'Guangxi',
  'Guizhou',
  'Hainan',
  'Hebei',
  'Heilongjiang',
  'Henan',
  'Hong Kong',
  'Hubei',
  'Hunan',
  'Inner Mongolia',
  'Jiangsu',
  'Jiangxi',
  'Jilin',
  'Liaoning',
  'Macau',
  'Ningxia',
  'Qinghai',
  'Shaanxi',
  'Shandong',
  'Shanghai',
  'Shanxi',
  'Sichuan',
  'Tianjin',
  'Tibet',
  'Xinjiang',
  'Yunnan',
  'Zhejiang',
  'Anhui',
  'Beijing',
  'Chongqing',
  'Fujian',
  'Gansu',
  'Guangdong',
  'Guangxi',
  'Guizhou',
  'Hainan',
  'Hebei